In [1]:
import pandas as pd
from datetime import date
import numpy as np

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('employees_secret.json', scope)
client = gspread.authorize(creds)
sheet = client.open('Stocks').sheet1
stock_details = sheet.get_all_records()
stock_metrics=pd.DataFrame(stock_details)

sheet = client.open('Stocks').get_worksheet(1)
stock_52_trends_l = sheet.get_all_records()
stock_52_trends=pd.DataFrame(stock_52_trends_l)
stock_52_trends = stock_52_trends.drop(stock_52_trends.columns[-1],axis=1)
stock_52_trends.replace(0,stock_52_trends.mean(axis=0),inplace=True)

sheet = client.open('Stocks').get_worksheet(3)
sector_mapping = sheet.get_all_records()
sector_mapping=pd.DataFrame(sector_mapping)
#sector_mapping

#print(telemedicine)

In [4]:
#Append today's price to the stock trends sheet
from datetime import date
today = str(date.today().strftime("%d-%m-%Y"))
today
stock_52_trends.insert(loc=1, column=today, value=stock_metrics['price'])


#stock_52_trends_l.describe()
k=stock_52_trends[stock_52_trends.Symbol=='AUROPHARMA']

In [366]:
k

,Symbol,29-12-2019,23-12-2019,22-12-2019,21-12-2019,20-12-2019,19-12-2019,18-12-2019,17-12-2019,16-12-2019,...,02-01-2019,01-01-2019,31-12-2018,30-12-2018,29-12-2018,28-12-2018,27-12-2018,26-12-2018,25-12-2018,24-12-2018
55,AUROPHARMA,462.5,465.05,468.15,468.15,468.15,470.4,464.2,454.0,448.75,...,715.9,724.35,733.15,732.95,732.95,732.95,723.95,711.85,718.5,718.5


In [5]:
stock_52_trends_l=stock_52_trends.melt(['Symbol'], var_name='Date', value_name='price')
print(stock_52_trends_l.dtypes)
stock_52_trends_l['Date'] = pd.to_datetime(stock_52_trends_l['Date'],format='%d-%m-%Y')
stock_52_trends_l.sort_values(by=['Date'], ascending=True, inplace=True)

type(stock_52_trends_l)
stock_52_trends_l.describe()

type(stock_52_trends_l['Date'])

#stock_52_trends_l=stock_52_trends_l[stock_52_trends_l['price']!=0]

#stock_52_trends_l['price']=stock_52_trends_l['price']+0.0001
stock_52_trends_l['price']=np.log(stock_52_trends_l['price'])

#Add sector info to stock trends sheet
stock_52_trends_l = pd.merge(left=stock_52_trends_l,right=sector_mapping,left_on='Symbol', right_on='Symbol')

#print(stock_52_trends_l.dtypes)


stock_52_trends_l.to_csv('stock_52_trends_l.csv')


Symbol     object
Date       object
price     float64
dtype: object


In [376]:
auro=stock_52_trends_l[stock_52_trends_l.Symbol=='AUROPHARMA']
auro


In [6]:
#type(stock_52_trends_l.Symbol.unique()[0])
#stock_52_trends.iloc[:, [30]]
stock_52_trends_l.sort_values(by=['Date'], ascending=True, inplace=True)
stock_52_trends_l.describe()         



,price
count,37332.000000
mean,6.144071
std,1.716490
min,0.285179
25%,5.301313
50%,6.201675
75%,7.296430
max,11.118400


In [7]:
#Transpose and format the date column
#stock_52_trends_l=stock_52_trends.melt(['Symbol'], var_name='Date', value_name='price')
#stock_52_trends_l
today=stock_52_trends.iloc[:, [2]].T.squeeze()
month=stock_52_trends.iloc[:, [30]].T.squeeze()
qtr=stock_52_trends.iloc[:, [90]].T.squeeze()
half=stock_52_trends.iloc[:, [180]].T.squeeze()
year=stock_52_trends.iloc[:, [365]].T.squeeze()

mth_returns=(today-month)/month
qtr_returns=(today-qtr)/qtr
hly_returns=(today-half)/half
yly_returns=(today-year)/year



#stock_52_trends['mth_returns']


In [10]:
#stocks_all = pd.merge(left=stock_latest_price,right=stock_metrics,left_on='Symbol', right_on='Symbol')
#stock_metrics

In [8]:
#Calculate all metrics on stocks
cols = ['eps','high52','low52','pe','price']
stock_metrics[cols] = stock_metrics[cols].replace({0:np.nan})

stock_metrics['price_to_high'] = (stock_metrics['price']-stock_metrics['high52'])/stock_metrics['high52']
stock_metrics['price_to_low'] = (stock_metrics['price']-stock_metrics['low52'])/stock_metrics['low52']

stock_metrics['low_to_high'] = stock_metrics['low52']/stock_metrics['high52']
stock_metrics['mean']=stock_52_trends.sum(axis=1)/365
stock_metrics['sd']=stock_52_trends.std(axis = 1, skipna = True) 
stock_metrics['cv']=stock_metrics['sd']/stock_metrics['mean']
#stock_metrics

#returns
stock_metrics['mth_returns']=mth_returns
stock_metrics['qtr_returns']=qtr_returns
stock_metrics['hly_returns']=hly_returns
stock_metrics['yly_returns']=yly_returns

gp=stock_metrics.groupby('sector').agg({'pe': ['max','min','mean']}).reset_index()

gp.columns=['sector','pe_sector_max','pe_sector_min','pe_sector_mean']

stock_metrics_pe = pd.merge(left=stock_metrics,right=gp,left_on='sector', right_on='sector')

stock_metrics_pe.to_csv('stock_metrics_pe.csv',index=False)



In [9]:
#Output PE based datasets
undervalued=stock_metrics_pe.sort_values(by=['price_to_high'],inplace=False,ascending=False)
overvalued=undervalued.tail(10)
undervalued=undervalued.head(10)
#undervalued=undervalued.filter(['Symbol','price','low52','high52','price_to_high','cv',])
#overvalued=overvalued.filter(['Symbol','price','low52','high52','price_to_high','cv',])

undervalued.to_csv('undervalued.csv',index = False)
overvalued.to_csv('overvalued.csv',index = False)
#overvalued['price_to_high'] = overvalued['price_to_high'].map('{:,.2f}'.format)

overvalued
# ovrvalued=stock_metrics_pe.sort_values(by=['price_to_high'],inplace=False,ascending=True)
# overvalued=overvalued.head(10)
# overvalued=ovrvalued.filter(['Symbol','price','low52','high52','price_to_high','cv',])
#overvalued=undervalued.tail(10)
#overvalued.columns
#overvalued['price_to_high']
#undervalued

,Symbol,eps,high52,low52,pe,beta,marketcap,price,netprofit,sector,...,mean,sd,cv,mth_returns,qtr_returns,hly_returns,yly_returns,pe_sector_max,pe_sector_min,pe_sector_mean
41,GAIL,13.28,183.25,110.45,9.12,0.0,546178189318,121.10,5.989469e+10,Energy - Oil & Gas,...,165.593385,145.455799,0.878391,-0.035200,-0.101006,-0.228654,-0.317487,21.53,6.30,12.343333
14,M&M,48.70,810.00,502.55,10.91,0.0,633400725298,531.25,5.806422e+10,Automobile,...,620.061439,82.990299,0.133842,-0.023580,-0.052603,-0.203182,-0.332360,40.93,10.91,24.666667
37,NIACL,3.67,219.00,NaN,36.79,0.0,223387470123,135.00,6.072830e+09,Financial Services,...,152.971644,29.288454,0.191463,-0.041278,0.362346,-0.031607,-0.182051,93.62,8.18,38.876923
0,ZEEL,18.73,488.95,199.15,15.59,0.0,280745888608,292.00,1.800812e+10,Media & Entertainment,...,371.559518,129.893117,0.349589,4.258390,5.145521,3.691244,0.069349,15.59,12.39,13.990000
96,BHEL,2.17,78.85,41.30,20.03,0.0,151295640006,43.40,7.564782e+09,Industrial Manufacturing,...,61.286164,9.725973,0.158698,-0.175701,-0.118000,-0.396304,-0.383648,48.47,14.57,29.722500
22,AUROPHARMA,43.91,838.00,389.35,10.41,0.0,267862285751,457.30,2.572017e+10,Pharmaceuticals,...,634.972329,121.109352,0.190732,0.044000,-0.221283,-0.244251,-0.352749,110.49,10.41,32.475000
26,PEL,103.61,2855.83,1348.16,14.70,0.0,341421682250,1523.10,2.322546e+10,Pharmaceuticals,...,2055.372329,339.747907,0.165297,-0.064632,-0.045726,-0.161501,-0.287657,110.49,10.41,32.475000
84,UPL,NaN,4.00,1.31,NaN,0.0,7989951,1.35,0.000000e+00,Chemicals,...,23.540802,184.232240,7.826082,0.090226,-0.256410,-0.185393,-0.539683,65.77,65.77,65.770000
63,NSE:IDEA,-26.64,23.01,2.40,NaN,0.0,177009936415,6.20,-7.605717e+11,Telecommunication,...,1571.237705,107.374475,0.068338,-0.010403,0.053435,0.043294,0.069349,15.03,15.03,15.030000
70,YESBANK,-4.16,286.00,29.00,NaN,0.0,119744709295,47.00,-1.059868e+10,Banking,...,132.041644,73.881615,0.559533,-0.213777,0.018462,-0.545121,-0.723321,81.49,24.02,36.815000


In [397]:
# k=[{"label": "BAJFINANCE", "value": "BAJFINANCE"},{"label": "INDUSINDBK", "value": "INDUSINDBK"},{"label": "HDFC", "value": "HDFC"}]
# l={"label": "BAJFINANCE", "value": "BAJFINANCE"}
# type(l)

#dcc.Slider(min=stock_52_trends_l['Date'].min(), max=stock_52_trends_l['Date'].max(), value=stock_52_trends_l['Date'].min(), marks = {date.day:date.day for date in stock_52_trends_l['Date'].stock_52_trends_l.Date.unique()})
sector='Automobile'
filtered_stocks=stock_52_trends_l[stock_52_trends_l.sector==sector]
    
symlist=filtered_stocks.Symbol.unique()
optlist=[]
for i in range(len(symlist)):
    x={'label':symlist[i],'value':symlist[i]}
    optlist.append(x)
optlist

[{'label': 'EICHERMOT', 'value': 'EICHERMOT'},
 {'label': 'M&M', 'value': 'M&M'},
 {'label': 'TATAMOTORS', 'value': 'TATAMOTORS'},
 {'label': 'MARUTI', 'value': 'MARUTI'},
 {'label': 'HEROMOTOCO', 'value': 'HEROMOTOCO'},
 {'label': 'BAJAJ-AUTO', 'value': 'BAJAJ-AUTO'},
 {'label': 'MOTHERSUMI', 'value': 'MOTHERSUMI'},
 {'label': 'MRF', 'value': 'MRF'},
 {'label': 'ASHOKLEY', 'value': 'ASHOKLEY'},
 {'label': 'BOSCHLTD', 'value': 'BOSCHLTD'}]

In [145]:
from datetime import datetime as dt
#k=datetime.datetime(2009, 1, 5)
k=max(stock_52_trends_l['Date'])

start_date='2018/12/25'
start_date = dt.strptime(start_date.split(' ')[0], '%Y/%m/%d')
start_date

end_date='2019/06/25'
end_date = dt.strptime(end_date.split(' ')[0], '%Y/%m/%d')

filter_df=stock_52_trends_l[(stock_52_trends_l['Date']>=start_date)&(stock_52_trends_l['Date']<=end_date)]
#filter_df
 
start_date='2019-01-01T00:00:00'
new_dt = start_date[:10]
start_date = dt.strptime(new_dt.split(' ')[0], "%Y-%m-%d")
start_date

datetime.datetime(2019, 1, 1, 0, 0)

In [336]:
# #stock_metrics_pe.to_dict('records')
# seclist=stock_metrics_pe.sector.unique()
# optlist_sec=[]
# for i in range(len(seclist)):
#     x={'label':seclist[i],'value':seclist[i]}
#     optlist_sec.append(x)
# optlist_sec

stock_metrics_pe=pd.read_csv('stock_metrics_pe.csv')

undervalued=stock_metrics_pe.sort_values(by=['price_to_high'],inplace=False,ascending=False)
overvalued=undervalued.tail(10)
overvalued


stock_returns=stock_metrics_pe.filter(['Symbol','yieldpct','sector','mth_returns','qtr_returns','hly_returns','yly_returns'])

stock_returns

,Symbol,yieldpct,sector,mth_returns,qtr_returns,hly_returns,yly_returns
0,ZEEL,0,Media & Entertainment,4.261177,5.148778,3.693730,0.068957
1,SUNTV,0,Media & Entertainment,2.449545,2.482759,2.289329,1.923689
2,VEDL,0,Metals,0.029376,-0.083878,-0.173058,-0.263573
3,TATASTEEL,0,Metals,0.089591,0.284999,-0.088372,-0.099912
4,HINDALCO,0,Metals,0.082788,0.128212,0.029919,-0.015782
...,...,...,...,...,...,...,...
95,NSE:BEL,0,Industrial Manufacturing,-0.010414,0.053334,0.043038,0.068957
96,BHEL,0,Industrial Manufacturing,-0.175701,-0.118000,-0.396304,-0.383648
97,SIEMENS,0,Industrial Manufacturing,0.026340,0.016037,0.152430,0.479757
98,CONCOR,0,Services - Transport & Logistics,0.044333,-0.015732,0.006487,0.114484


In [3]:
import pandas as pd
chk=pd.read_csv('stock_metrics_pe.csv')
type(chk.columns)



pandas.core.indexes.base.Index